In [ ]:
# Importar librerias
import pandas as pd
import json
import numpy as np
from pandas import json_normalize
from textblob import TextBlob

1. Apertura y visualizacion primer archivo JSON.

In [ ]:
# Apertura del archivo output_steam_games
Df_Games = pd.read_json('./file_json/output_steam_games.json', lines= True)
# Visualización del DataFrame
Df_Games.head()

In [ ]:
# Información general de los datos cargados en el DataFrame
Df_Games.info()

In [ ]:
# Se valida los valores nulos 
Df_Games.isnull().sum()

In [ ]:
# Se eliminan las columnas que no seran utilizadas para el proceso de modelado
Df_Games.drop(['publisher','title','url','tags','reviews_url','specs','early_access',], axis=1, inplace=True)

In [ ]:
# Eliminación de Filas con todos sus Valores Nulos
Df_Games.dropna(inplace=True)
Df_Games.head()

In [ ]:
# Se visualizan los datos de la columna RELEASE DATE
unique_value = Df_Games['release_date'].unique()
for i in unique_value:
    print(i)

In [ ]:
# Se convierte la columna RELEASE DATE a tipo datetime
Df_Games['release_date'] = pd.to_datetime(Df_Games['release_date'], errors= 'coerce')
Df_Games['release_date']

In [ ]:
# Se extrae el año y se crea una nueva columna release_year
Df_Games['release_year'] = Df_Games['release_date'].dt.year.astype('Int64')
# Se elimina la columna RELEASE_DATE 
Df_Games = Df_Games.drop(columns=['release_date'])
# Se rellena con la media los valores faltantes de la columna release_year
Df_Games['release_year'] = Df_Games['release_year'].fillna(Df_Games['release_year'].median())

In [ ]:
# Se reinicia el índice del DataFrame después de realizar manipulaciones de datos para organizar y actualizar su estructura.
Df_Games.reset_index(drop=True, inplace=True)

In [ ]:
# Transformamos la columna 'genres' convirtiéndola en listas, desplegamos esas listas y luego utilizamos 'get_dummies' para asignar valores numéricos a cada elemento en esas listas.
Df_genres = pd.DataFrame(Df_Games['genres'].tolist())
Df_genres_obj = Df_genres.stack()
Df_genres_1 = pd.get_dummies(Df_genres_obj)
Df_genres_1 = Df_genres_1.groupby(level=[0]).sum()
genres = Df_genres_1 

In [ ]:
genres = Df_genres_1
Df_genres_1.head(1)

In [ ]:
# Resetear los índices de ambos DataFrames
Df_Games.reset_index(drop=True, inplace=True)
genres.reset_index(drop=True, inplace=True)

In [ ]:
# Combinamos dos DataFrames y eliminamos la columna de 'genres'.
Df_Games = pd.concat([Df_Games, genres],axis=1)
Df_Games.drop(columns=['genres'],inplace=True)
Df_Games.dropna(subset='Indie',inplace=True)

In [ ]:
# Convertir la columna PRICE a tipo numérico
Df_Games['price']= pd.to_numeric(Df_Games['price'], errors='coerce')
# Convertir la columna ID a tipo entero
Df_Games['id']= Df_Games['id'].astype('Int64')

In [ ]:
# Se guardan los datos limpios en un CSV para su posterior manipulación
Df_Games.to_csv('Output_steam_games_new.csv', index=False)

2. Apertura y visualizacion Segundo archivo JSON.

In [ ]:
# Apertura y visualización del archivo australian_users_items
with open ('./file_json/australian_users_items.json', 'r', encoding='utf-8') as file:
    data = file.readlines()
registros = [eval(line.strip()) for line in data]
Df_Items = pd.DataFrame(registros)
Df_Items.head()

In [ ]:
# La columna 'Items' está anidada, se procesa para extraer información específica y estructurarla en nuevas columnas
Df_Items['playtime_forever'] = Df_Items['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
Df_Items['playtime_2weeks'] = Df_Items['items'].apply(lambda x: x[0].get('playtime_2weeks') if len(x) > 0 else 0)
Df_Items['id'] = Df_Items['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
Df_Items['id'] = Df_Items['id'].astype(int)
Df_Items['playtime_forever'] = Df_Items['playtime_forever'].astype(int)
Df_Items[']playtime_2weeks'] = Df_Items['playtime_2weeks'].astype(int)
Df_Items.head()

In [ ]:
# Se eliminan las columnas que no seran utilizadas para el proceso de modelado
Df_Items.drop(['items','items_count','user_url'], axis=1, inplace=True)

In [ ]:
# Verificar registros duplicados 
duplicados = Df_Items.duplicated()
# Eliminar registros duplicados
Df_Items = Df_Items.drop_duplicates()
# Eliminar registros nulos
Df_Items = Df_Items.dropna(how='all')

In [ ]:
# Eliminar los registros con valor 0 en la columna playtime_forever
Df_Items = Df_Items.drop(Df_Items[Df_Items['playtime_forever'] == 0].index)
Df_Items.tail()

In [ ]:
# Se guardan los datos limpios en un CSV para su posterior manipulación
Df_Items.to_csv('australian_user_items_new.csv', index=False)

3. Apertura y visualizacion Tercer archivo JSON.

In [ ]:
# Apertura y visualización del archivo australian_user_reviews
user_reviews = []
with open ('./file_json/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data = file.readlines()
registros = [eval(line.strip()) for line in data]
Df_reviews = pd.DataFrame(registros)
Df_reviews.head()

In [ ]:
# La columna 'reviews' tiene datos anidados, por lo que es necesario desglosarla para generar nuevas filas para cada elemento contenido en ella.
Df_reviews = Df_reviews.explode('reviews', ignore_index=True)
Df_reviews_new = json_normalize(Df_reviews['reviews'])

In [ ]:
# Se combinan los DataFrames
Df_reviews = Df_reviews.join(Df_reviews_new)
Df_reviews.head()

In [ ]:
# Se eliminan las columnas que no seran utilizadas para el proceso de modelado
Df_reviews.drop(['user_url','reviews', 'funny', 'last_edited', 'helpful'], axis=1, inplace=True)

In [ ]:
# Se identifican registros duplicados y los eliminamos
duplicados = Df_reviews.duplicated()
Df_reviews = Df_reviews.drop_duplicates()
# Se eliminan los registros nulos
Df_reviews = Df_reviews.dropna(how='all')

In [ ]:
# Se extrae el año de la columna posted
# Me aseguro que 'posted' sea de tipo datetime
Df_reviews['posted'] = pd.to_datetime(Df_reviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')
# Crea la columna 'year' a partir de 'posted'
Df_reviews['year'] = Df_reviews['posted'].dt.year.astype('Int64')
Df_reviews.head()

In [ ]:
#Se eliminan los nulos de year
Df_reviews.dropna(subset='year', inplace=True)

In [ ]:
#Se crea una función para detectar el sentimiento
def sentimiento(texto):
        analisis = TextBlob(texto)
        #Se obtiene la polaridad
        polaridad = analisis.sentiment.polarity
        if polaridad < 0:
            return 0  # Negativo
        elif polaridad > 0:
            return 2  # Positivo
        else:
            return 1 # Neutro

In [ ]:
# Se crea la columna sentiment
Df_reviews['sentiment'] = Df_reviews['review'].apply(sentimiento)
Df_reviews.head()

In [ ]:
# Se guardan los datos limpios en un CSV para su posterior manipulación
Df_reviews.to_csv('australian_user_reviews_new.csv', index=False)

In [34]:
# Se unen los DataFrames 
# Se unen los dataframe games e items por su item_id. Y a su vez se une con el df reviews por el user_id
Df1 = pd.merge(Df_Games, Df_Items,on='id')
Df_General = pd.merge(Df1, Df_reviews,on='user_id')
# Se guarda el archivo del DataFrame general
Df_General.to_csv('Df_General.csv', index=False)
Df_General.to_parquet('Df_General.parquet', index=False)